In [2]:
from __future__ import division
import pandas as pd
import matplotlib.pyplot as plt
import sklearn as sk
import numpy as np

In [5]:
try:
    train_data = pd.read_csv("numerai_training_data.csv")
    test_data = pd.read_csv("numerai_tournament_data.csv")
except:
    try:
        train_data = pd.read_csv("numerai_training_data_limited.csv")
        test_data = pd.read_csv("numerai_tournament_data_limited.csv")
    except:
        print "error - pure sadness. you have no data files in this directory"
    

In [6]:
train_data.head()

,feature1,feature2,feature3,feature4,feature5,feature6,feature7,feature8,feature9,feature10,...,feature42,feature43,feature44,feature45,feature46,feature47,feature48,feature49,feature50,target
0,0.360233,0.400324,0.376192,0.625575,0.578128,0.456694,0.366812,0.354266,0.498456,0.539569,...,0.538909,0.447431,0.544526,0.574966,0.382544,0.465450,0.336393,0.577600,0.716399,1.0
1,0.524941,0.562669,0.594656,0.691748,0.780364,0.507443,0.473722,0.499549,0.668819,0.289693,...,0.422837,0.174319,0.511849,0.681012,0.506451,0.428276,0.408187,0.721181,0.738231,1.0
2,0.375577,0.343138,0.442795,0.549371,0.238039,0.730104,0.383640,0.439865,0.333091,0.735542,...,0.423726,0.591998,0.482158,0.365507,0.372470,0.337441,0.368906,0.576928,0.536500,1.0
3,0.419479,0.284885,0.299801,0.466293,0.519207,0.471024,0.601611,0.442310,0.486457,0.628480,...,0.660233,0.443502,0.578453,0.544214,0.541356,0.452928,0.353136,0.591958,0.620773,1.0
4,0.563124,0.485997,0.531946,0.532138,0.472720,0.450243,0.746844,0.242323,0.767916,0.497159,...,0.526667,0.443528,0.414979,0.417109,0.544803,0.480563,0.626479,0.438063,0.562606,0.0


In [7]:
#dimensions of data
train_data.shape

(260766, 51)

In [8]:
#rerunning the lines of code below will overwrite data_limited files

"""
#determining number of rows for limited dataset to upload to github (max 100 mb)
github_max = 100
file_size = 120
max_rows = int(round(github_max/file_size*data.shape[0],-3))

train_data_limited = train_data[0:max_rows]
test_data_limited = test_data[0:max_rows]

train_data_limited.to_csv('numerai_training_data_limited.csv')
test_data_limited.to_csv('numerai_tournament_data_limited.csv')
print "The first "+ str(max_rows) +" rows of the data were added to limited datasets"
"""

'\n#determining number of rows for limited dataset to upload to github (max 100 mb)\ngithub_max = 100\nfile_size = 120\nmax_rows = int(round(github_max/file_size*data.shape[0],-3))\n\ntrain_data_limited = train_data[0:max_rows]\ntest_data_limited = test_data[0:max_rows]\n\ntrain_data_limited.to_csv(\'numerai_training_data_limited.csv\')\ntest_data_limited.to_csv(\'numerai_tournament_data_limited.csv\')\nprint "The first "+ str(max_rows) +" rows of the data were added to limited datasets"\n'

In [9]:
first_row = train_data[0:1]
first_row

,feature1,feature2,feature3,feature4,feature5,feature6,feature7,feature8,feature9,feature10,...,feature42,feature43,feature44,feature45,feature46,feature47,feature48,feature49,feature50,target
0,0.360233,0.400324,0.376192,0.625575,0.578128,0.456694,0.366812,0.354266,0.498456,0.539569,...,0.538909,0.447431,0.544526,0.574966,0.382544,0.46545,0.336393,0.5776,0.716399,1.0


In [10]:
first_column = train_data['feature1']
first_column

0         0.360233
1         0.524941
2         0.375577
3         0.419479
4         0.563124
5         0.732232
6         0.500674
7         0.496982
8         0.516010
9         0.325679
10        0.615414
11        0.473327
12        0.275815
13        0.279551
14        0.652488
15        0.532997
16        0.443737
17        0.492230
18        0.538208
19        0.433338
20        0.692012
21        0.513038
22        0.379924
23        0.632134
24        0.382237
25        0.427203
26        0.575131
27        0.442043
28        0.571538
29        0.418257
            ...   
260736    0.364066
260737    0.398505
260738    0.543030
260739    0.399012
260740    0.514064
260741    0.230285
260742    0.514187
260743    0.422090
260744    0.701840
260745    0.470094
260746    0.503395
260747    0.322674
260748    0.540157
260749    0.451574
260750    0.461459
260751    0.354464
260752    0.734633
260753    0.569964
260754    0.340725
260755    0.324002
260756    0.908379
260757    0.

In [11]:
#Plot out what feature1 looks like
plt.hist(first_column, bins=50)
plt.title('Distribution of Feature1')
plt.ylabel('Frequency')

plt.show()